# Rcpp入門 その2

「オリジナルのR」、「RcppArmadillo」、「RcppEigen」で処理速度の比較をしてみたいと思います。

「RcppArmadillo」、「RcppEigen」を使うのは、恐らく以下のような時だと思います。

* forループを多用しなくてはいけない時
* ライブラリ固有の関数を使いたい時

[Rcpp入門](Rcpp_Getting_Started.ipynb)の関数を、1固定ではなく、行番号+列番号とした関数を作成して比較します。

## Rで作成した関数

In [11]:
PlusIJ_R <- function(src)
{
    num_row <- nrow(src)
    num_col <- ncol(src)
    
    for(r in 1:num_row)
    {
        for(c in 1:num_col)
        {
            src[r, c] = src[r, c] + r + c
        }
    }
    
    return(src)
}

## RcppArmadilloで作成した関数

In [12]:
# パッケージの読込み
library(Rcpp)
library(RcppArmadillo)

# C+11にてコンパイルします
Sys.setenv("PKG_CXXFLAGS"="-std=c++11")

In [13]:
# C++のコード
code <- '
arma::mat PlusIJ_Arma(arma::mat src)
{
    int num_row = src.n_rows;
    int num_col = src.n_cols;
    
    for(int r = 0; r < num_row; r++)
    {
        for(int c = 0; c < num_col; c++)
        {
            src(r, c) = src(r, c) + r + 1 + c + 1;
        }
    }

    return src;
}
'

# コンパイルとリンクを行います（少し時間がかかります）
cppPlusIJ_Arma <- cppFunction(code, depends="RcppArmadillo")

## RcppEigenで作成した関数

In [14]:
# パッケージの読込み
library(Rcpp)
library(RcppEigen)

# C+11にてコンパイルします
Sys.setenv("PKG_CXXFLAGS"="-std=c++11")

In [15]:
# C++のコード
code <- '
Eigen::MatrixXd PlusIJ_Eigen(Eigen::MatrixXd src)
{
    int num_row = src.rows();
    int num_col = src.cols();
    
    for(int r = 0; r < num_row; r++)
    {
        for(int c = 0; c < num_col; c++)
        {
            src(r, c) = src(r, c) + r + 1 + c + 1;
        }
    }

    return src;
}
'

# コンパイルとリンクを行います（少し時間がかかります）
cppPlusIJ_Eigen <- cppFunction(code, depends="RcppEigen", includes=c("#include <RcppEigen.h>"))

## 比較

元データを作成します。

In [40]:
src <- matrix(1:100000000, nrow=10000)

**Rで作成した関数**

In [31]:
t <- proc.time()

dst_r <- PlusIJ_R(src)

print(proc.time() - t)

   user  system elapsed 
  14.26    0.16   14.44 


**RcppArmadilloで作成した関数**

In [36]:
t <- proc.time()

dst_arma <- cppPlusIJ_Arma(src)

print(proc.time() - t)

   user  system elapsed 
   1.91    0.57    2.46 


**RcppEigenで作成した関数**

In [39]:
t <- proc.time()

dst_eigen <- cppPlusIJ_Eigen(src)

print(proc.time() - t)

   user  system elapsed 
   1.87    0.41    2.28 


**結果のチェック**

In [34]:
print("RcppArmadilloで作成した関数チェック")
all(dst_r == dst_arma)
print("RcppEigenで作成した関数チェック")
all(dst_r == dst_eigen)

[1] "RcppArmadilloで作成した関数チェック"


[1] TRUE

[1] "RcppEigenで作成した関数チェック"


[1] TRUE

今回の結果としては、以下の順位になりました。

1. 「RcppArmadillo」 ≒ 「RcppEigen」
1. 「オリジナルのR」

forループを多用するコードになる場合は積極的に、「RcppArmadillo」や「RcppEigen」を使うべきだと思いました。

「RcppArmadillo」と「RcppEigen」のどちらを利用するかは、悩みどころです。

----
[ホームへ](https://waku-take-a.github.io/index.html)  
[↑Rの記事Topへ](https://github.com/WAKU-TAKE-A/RTips/blob/master/README.md)  

※  
上記のipynbファイル等は、[こちらに](https://github.com/WAKU-TAKE-A/RTips)あります。  
解凍後、`The jupyter notebook`にアップロードすれば、ローカルな環境で実行することも、編集することもできます。

<(_ _)>